In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install -U langchain-openai

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [3]:
!mkdir pdfs

mkdir: pdfs: File exists


In [4]:
loader=PyPDFDirectoryLoader("pdfs")

In [5]:
data=loader.load()

In [6]:
data[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [8]:
text_chunks = text_splitter.split_documents(data)

In [9]:
len(text_chunks)

91

In [10]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [11]:
print(text_chunks[1].page_content)

aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,


In [12]:
print(text_chunks[2].page_content)

based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,


In [13]:
print(text_chunks[90].page_content)

just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention
at layer 5 of 6. The heads clearly learned to perform different tasks.
15


In [14]:
import os
os.environ["OPENAI_API_KEY"]="YOUR_OPENAI_API_KEY"

In [15]:
embedding=OpenAIEmbeddings()

In [16]:
len(embedding.embed_query("How are you"))

1536

In [17]:
len(embedding.embed_query("hi i am fine"))

1536

In [18]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'YOUR_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'YOUR_ENVIRONMENT')

In [19]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [20]:
from langchain_community.vectorstores import Pinecone as PineconeLang
from pinecone import Pinecone as PineconeClient

pc = PineconeClient(
    api_key = 'PINECONE_API_KEY'
)

In [22]:
index_name = "testing" # put in the name of your pinecone index here

In [23]:
index = pc.Index(index_name)

In [24]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00091,
 'namespaces': {'': {'vector_count': 91}},
 'total_vector_count': 91}

## Create Embeddings for each of the Text Chunk

In [25]:
docsearch = PineconeLang.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [26]:
docsearch

In [27]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7fc4fa23b580>)

In [28]:
query= "what is attention?"

In [29]:
docs = docsearch.similarity_search(query)

In [30]:
docs

[Document(page_content='described in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been\nused successfully in a variety of tasks including reading comprehension, abstractive summarization,\ntextual entailment and learning task-independent sentence representations [4, 27, 28, 22].'),
 Document(page_content='just\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:\nFull attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5\nand 6. Note that the attentions are very sharp for this word.\n14'),
 Document(page_content='the input or output sequences [ 2,19]. In all but a few cases [ 27], however, such attention mechanisms\nare used in conjunction with a recurrent n

In [31]:
llm = OpenAI()

/Users/prashantharipirala/opt/anaconda3/envs/testopenai/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [32]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [33]:
query= "what is attention?"

In [34]:
qa.run(query)

/Users/prashantharipirala/opt/anaconda3/envs/testopenai/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Attention is a mechanism used in natural language processing to relate different positions of a single sequence in order to compute a representation of the sequence. It has been used successfully in various tasks such as reading comprehension, summarization, and sentence representation.'

In [36]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt:  What is attention?


Answer:  Attention is an attention mechanism that relates different positions of a single sequence in order to compute a representation of the sequence. It has been used successfully in tasks such as reading comprehension, summarization, and sentence representation learning.


Input Prompt:  who invented attention?


Answer:  Noam


Input Prompt:  what are the benefits of attention, and give me any related metrics.


...

Answer: 

Attention and self-attention are both mechanisms used in natural language processing tasks to focus on specific parts of a sequence. However, the main difference is that attention relates different positions of two different sequences, while self-attention relates different positions of a single sequence. In other words, attention is used to align two sequences, while self-attention is used to compute a representation of a single sequence. Additionally, self-attention has been shown to be successful in a variety of tasks, while attention is typically used in more specific tasks such as machine translation.


Input Prompt:  exit


Exiting


SystemExit: 

/Users/prashantharipirala/opt/anaconda3/envs/testopenai/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
